# Libraries

In [29]:
import os
import random
import numpy as np
import pandas as pd
import unicodedata
from huggingface_hub import HfApi

pd.set_option('max_colwidth', None)

# Config

In [30]:
seed = 42
lang = 'sun'

split = 'train' 
raw_data_path = f'data/public_data_dev/track_a/{split}/sun.csv'
# raw_data_path = 'data/back_translated_data_raw/track_a/sun/dev_final.csv'
preprocessed_data_dir = './data/preprocessed_data_raw/'

hf_data_id = 'alxxtexxr/SemEval2025-Task11-Dataset'
hf_data_config = lang + '_back_translated_dev'
print("Hugging Face dataset config:", hf_data_config)

save_dir = os.path.join(preprocessed_data_dir, 'track_a', hf_data_config)
save_path = os.path.join(save_dir, f'{split}.csv')

Hugging Face dataset config: sun_back_translated_dev


In [31]:
def set_seed(seed):
    # Set random seed for NumPy
    np.random.seed(seed)

    # Optionally set random seed for sklearn and Python's own random module
    random.seed(seed)

    # Set random seed for os
    os.environ['PYTHONHASHSEED'] = str(seed)

    print(f"Random seed set to: {seed}")

set_seed(seed)

Random seed set to: 42


# Data

## Load Data

In [32]:
df = pd.read_csv(raw_data_path)
print("Raw DF length:", len(df))
df.head()

Raw DF length: 924


,id,text,anger,disgust,fear,joy,sadness,surprise
0,sun_train_track_a_00001,Kumaha mang fiksi engke ka sabilulungan mang salam bandung soreang,0,0,0,1,0,1
1,sun_train_track_a_00002,tapi domba anakan namah lain ku kuring mreun. 😂 ku domba dai nyh,0,0,0,1,0,1
2,sun_train_track_a_00003,Aduh mang naha bet kudu penting di upload ma😂😂,0,0,0,1,0,0
3,sun_train_track_a_00004,"pokonamah nuhun sabandungeun , kita terus tingkatkeun skill babarengan mang :) KAPAN NIKAH ? :(",0,0,0,1,1,0
4,sun_train_track_a_00005,Kang eta teu isin?? Apa emng tukang ngisinkeun wkwk😅,0,0,0,1,0,1


## Preprocess Data

In [33]:
# df['text'] = df['text'].str.replace(r'(?<!menit\s)\b\d{1,2}:\s?\d{2}\b', '', regex=True) # Remove timestamps

# # # df['text'] = df['text'].str.replace(r'^#\w+', '', regex=True) # Remove hashtags at the start of the text
# df['text'] = df['text'].str.replace(r'#(\w+)', r'\1', regex=True) # Remove # from the remaining hashtags

# # Convert dialogue formats to use ':'
# df['text'] = df['text'].str.replace(r'<(\w+)>', r'\1:', regex=True)
# df['text'] = df['text'].str.replace(r'([A-Z])-', r'\1:', regex=True)
# df['text'] = df['text'].str.replace(r'([A-Z]) -', r'\1:', regex=True)
# df['text'] = df['text'].str.replace(r'([A-Z])=', r'\1:', regex=True)
# df['text'] = df['text'].str.replace(r'([A-Z]) =', r'\1:', regex=True)

# # Replace dialogue names
# df['text'] = df['text'].str.replace(r'\b[Qq]:', 'Batur:', regex=True)
# df['text'] = df['text'].apply(lambda x: x.replace('A:', 'Urang:') if 'Batur:' in x else x)

# df['text'] = df['text'].str.replace(r'\b[Aa]:', 'Batur:', regex=True)
# df['text'] = df['text'].str.replace(r'\b[Bb]:', 'Urang:', regex=True)

# df['text'] = df['text'].str.replace(r'-(?=\w*[0-9])\w+', '', regex=True) # Remove patterns like -ic3rd4ow4d
# # # df = df[df['text'].str.count(r'\s+') > 0] # Remove rows with only one word

# df['text'] = df['text'].str.strip()
# df['text'] = df['text'].str.lstrip(',.!:$%^&*+') # Remove leading punctuation marks (comma, dot, etc.)
# df['text'] = df['text'].str.strip()

df.head()

,id,text,anger,disgust,fear,joy,sadness,surprise
0,sun_train_track_a_00001,Kumaha mang fiksi engke ka sabilulungan mang salam bandung soreang,0,0,0,1,0,1
1,sun_train_track_a_00002,tapi domba anakan namah lain ku kuring mreun. 😂 ku domba dai nyh,0,0,0,1,0,1
2,sun_train_track_a_00003,Aduh mang naha bet kudu penting di upload ma😂😂,0,0,0,1,0,0
3,sun_train_track_a_00004,"pokonamah nuhun sabandungeun , kita terus tingkatkeun skill babarengan mang :) KAPAN NIKAH ? :(",0,0,0,1,1,0
4,sun_train_track_a_00005,Kang eta teu isin?? Apa emng tukang ngisinkeun wkwk😅,0,0,0,1,0,1


In [34]:
def contains_non_ascii(text):
    try:
        text.encode('ascii')
    except UnicodeEncodeError:
        return True
    return False

print("Total data with non-ASCII chars:", int(df['text'].apply(contains_non_ascii).sum()))

Total data with non-ASCII chars: 375


In [35]:
def normalize_to_ascii(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')

# def normalize_to_ascii(text):
#     # Normalize the text to decompose accented characters
#     text = unicodedata.normalize('NFKD', text)
#     # Remove only combining marks (like accents) while keeping emojis and other symbols
#     text = ''.join(char for char in text if not unicodedata.combining(char))
#     return text

# Normalize to ASCII equivalents
df['text'] = df['text'].apply(normalize_to_ascii)
print("Total data with non-ASCII chars (after normalizing them):", int(df['text'].apply(contains_non_ascii).sum()))

df.head()

Total data with non-ASCII chars (after normalizing them): 0


,id,text,anger,disgust,fear,joy,sadness,surprise
0,sun_train_track_a_00001,Kumaha mang fiksi engke ka sabilulungan mang salam bandung soreang,0,0,0,1,0,1
1,sun_train_track_a_00002,tapi domba anakan namah lain ku kuring mreun. ku domba dai nyh,0,0,0,1,0,1
2,sun_train_track_a_00003,Aduh mang naha bet kudu penting di upload ma,0,0,0,1,0,0
3,sun_train_track_a_00004,"pokonamah nuhun sabandungeun , kita terus tingkatkeun skill babarengan mang :) KAPAN NIKAH ? :(",0,0,0,1,1,0
4,sun_train_track_a_00005,Kang eta teu isin?? Apa emng tukang ngisinkeun wkwk,0,0,0,1,0,1


## Save Preprocessed Data

In [36]:
!mkdir -p $save_dir

df.to_csv(save_path, index=False)

print("Saved to:", save_path)

Saved to: ./data/preprocessed_data_raw/track_a/sun_back_translated_dev/train.csv


## Upload Preprocessed Data to Hugging Face

In [37]:
hf_api = HfApi()
hf_api.upload_folder(
    repo_id=hf_data_id,
    repo_type='dataset',
    folder_path=save_dir,
    path_in_repo=os.path.join('preprocessed_data_raw/track_a', hf_data_config),
)

CommitInfo(commit_url='https://huggingface.co/datasets/alxxtexxr/SemEval2025-Task11-Dataset/commit/052594e61014022a7ac584ffd6a88c298119c724', commit_message='Upload folder using huggingface_hub', commit_description='', oid='052594e61014022a7ac584ffd6a88c298119c724', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/alxxtexxr/SemEval2025-Task11-Dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='alxxtexxr/SemEval2025-Task11-Dataset'), pr_revision=None, pr_num=None)